## Data processing

In [ ]:
!pip install -q bitsandbytes peft datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00


In [ ]:
!pip install accelerate

In [ ]:
pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [ ]:
import os
import torch
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel
from datasets import Dataset, DatasetDict
import pandas as pd
import bitsandbytes as bnb
from transformers import AdamW

In [ ]:
file_path = 'train.csv'
df = pd.read_csv(file_path, engine='python' , on_bad_lines='skip')

In [ ]:
df

,original_source,target,source,source_links,optimal_choice
0,Vatican_City,Ancient_Egypt,Vatican_City,"['Latin', 'Demographics of Vatican City', 'Lat...",English_language
1,Tibet,Pluto,China,"['SOS Children in China', ""People's Republic o...",Africa
2,Toy_Story,Cocoa,New_Zealand,"['Wellington', 'Auckland', 'List of countries ...",Netherlands
3,Jew,Manifest_Destiny,Jew,"['Hebrew language', 'Judaism', 'Ethnic group',...",United_States
4,Monkey,Adolf_Hitler,Monkey,"['Ape', 'Chimpanzee', 'Fruit', 'Seed', 'Flower...",German_language
...,...,...,...,...,...
18633,Gulf_of_Mexico,European_Union,United_Nations,"['New York City', 'Arabic language', 'Chinese ...",United_Kingdom
18634,Body_mass_index,Toy_Story,Cinema_of_the_United_States,"['American popular music', '20th century', 'Mu...",Star_Wars_Episode_IV__A_New_Hope
18635,London_Zoo,CPU_cache,Technology,"['20th century', 'Space exploration', 'Science...",Integrated_circuit
18636,Iguanodon,Halloween,Autumn,"['Season', 'Tropics', 'Season', 'Temperate', '...",Halloween


In [ ]:
df['text'] = "<s>[INST]" + 'Given the current Wikipedia page we are on : '+ df['source'] + ' the original wikipedia page from where we started' + df['original_source'] + ' the proposed links to other Wikipedia pages are :' + df['source_links'] + ' In order to arrive the fastest, in terms of the number of Wikipedia pages visited, to the Wikipedia page :'+ df['target']+ ' On which link should we click on ?' "[/INST]" +  df["optimal_choice"] + '</s>'
df_clean =df.drop(['source','target','original_source' ,'source_links',	'optimal_choice'],axis=1)

In [ ]:
import pyarrow.dataset as ds
import pyarrow as pa

In [ ]:
#Convert to dataset object
dataset = ds.dataset(pa.Table.from_pandas(df_clean).to_batches())
dataset = Dataset(pa.Table.from_pandas(df_clean))

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 18638
})

#Test Model 1

In [ ]:
!pip install -U bitsandbytes
!pip install transformers==4.36.2
!pip install -U peft
!pip install -U accelerate
!pip install -U trl
!pip install datasets==2.16.0
!pip install sentencepiece

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset
import re

In [ ]:
base_model = "mistralai/Mistral-7B-Instruct-v0.2"
new_model = "Enlighten_Instruct"

In [ ]:
# Load base model
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        load_in_4bit=True,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.bos_token, tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

('<s>', '</s>')

In [ ]:
from google.colab import userdata
secret_hf = userdata.get('HUGGINGFACE_TOKEN')
!huggingface-cli login --token $secret_hf

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=1,
    learning_rate=2e-4,
    weight_decay=0.01,
    fp16=False,
    bf16=False,
    group_by_length=True,
    lr_scheduler_type="constant",
)

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/18638 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.459900
2,1.204500
3,1.254200
4,1.105100
5,1.079600
6,0.949600
7,1.035000
8,0.881300
9,0.746900
10,1.277600


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.459900
2,1.204500
3,1.254200
4,1.105100
5,1.079600
6,0.949600
7,1.035000
8,0.881300
9,0.746900
10,1.277600


In [ ]:
trainer.model.save_pretrained(new_model)
model.config.use_cache = True
model.eval()

NameError: name 'trainer' is not defined

# Test Model 2

In [ ]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2",
    load_in_8bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)
print(model)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
   

In [ ]:
model = prepare_model_for_kbit_training(model)
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.pad_token = "<pad>"

In [ ]:
CUTOFF_LEN = 768
LORA_R = 8
LORA_ALPHA = 2 * LORA_R
LORA_DROPOUT = 0.1

In [ ]:
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj"
                    , "down_proj", "lm_head"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

In [ ]:
def print_trainable_parameters(m):
    trainable_params = sum(p.numel() for p in m.parameters() if p.requires_grad)
    all_params = sum(p.numel() for p in m.parameters())
    print(f"trainable params: {trainable_params} || all params: {all_params} || trainable%: {100 * trainable_params / all_params}")

print_trainable_parameters(model)

trainable params: 21260288 || all params: 7262992384 || trainable%: 0.2927207805812288


In [ ]:
def generate_prompt(user_query):  #The prompt format is taken from the official Mistral huggingface page
  if user_query["source"] is not None and user_query["source"] is not None:
      p =  "<s>[INST]" + 'Given the current Wikipedia page we are on : '+ user_query['source'] + ' the original wikipedia page from where we started' + user_query['original_source'] + ' the proposed links to other Wikipedia pages are :' + user_query['source_links'] + ' In order to arrive the fastest, in terms of the number of Wikipedia pages visited, to the Wikipedia page :'+ user_query['target']+ ' On which link should we click on ?' "[/INST]" +  user_query["optimal_choice"] + '</s>'
      return p
  else:
    p = " [INST]" + "Hello" + "[/INST]" +  "Hello" + ""
    return p

In [ ]:
def tokenize(prompt):
    if prompt is None:
        prompt = ''
    return tokenizer(
        prompt + tokenizer.eos_token,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding="max_length"
    )

In [ ]:
train_data = dataset_train['train']

In [ ]:
train_data = train_data.shuffle().map(
    lambda x: tokenize(
        generate_prompt(x) if generate_prompt(x) is not None else ''
    )
)

Map:   0%|          | 0/3873 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=6,    # 3 or 6 is good
        learning_rate=1e-4,
        logging_steps=2,
        optim= 'adamw_bnb_8bit',
        save_strategy="epoch",
        output_dir="mistral-lora-instruct-toxic"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False

In [ ]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 186.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 159.06 MiB is free. Process 10725 has 14.59 GiB memory in use. Of the allocated memory 14.08 GiB is allocated by PyTorch, and 387.01 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Test
Prompt the model and look at wether it gives appropriated outputs

In [ ]:
prompt = '1 + 1 = ?'
model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

NameError: name 'tokenizer' is not defined

In [ ]:
model.to(device)
generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

AttributeError: 'list' object has no attribute 'to'